##### Azure Event Hubs <br>  -> 1 Stage layer Delta lake <br> -> 2 split & flattened message type delta table Raw layer <br> -> 3 apply transformation rules Curated delta lake layer

# Structured Streaming with Azure EventHubs 
Maven library with coordinate `com.microsoft.azure:azure-eventhubs-spark_2.12:2.3.21`
</br>Python library `azure-eventhub`

In [0]:
import time
import os
import uuid
import datetime
import random
import json
import random
import datetime
import uuid
def get_customer_transaction_Stream():
    x = random.randint(1,2)
    x = x*100 
    y = x+10 
    data = {}
    now = datetime.datetime.now()
    str_now = now.isoformat()
    data['transaction_id'] = str(uuid.uuid4())
    data['store_number'] = random.randint(101,200)
    data['customer_number'] = random.randint(21000,29999)
    data['product'] = random.choice(['Burger','Pizza','Pasta', 'IceCream','Juice','Chips'])
    data['sold_quantity'] = random.randint(1,5)
    data['customer_total_spent'] = random.randint(20,100)
    data['customer_state'] = random.choice(['NSW','VIC', 'QLD', 'WA','SA','NT'])
    data['transaction_time'] = str_now
    return data

from azure.eventhub import EventHubProducerClient, EventData, TransportType 

CONNECTION_STR = "Endpoint=sb://zehubname1.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=xxxxxxxxxxxxxx=;EntityPath=sales_topic"
EVENTHUB_NAME = "sales_topic"

# This script simulates the production of events transport_type 2 = AmqpOverWebsocket
# Create a producer client to produce and publish events to the event hub.
producer = EventHubProducerClient.from_connection_string(conn_str = CONNECTION_STR, eventhub_name = EVENTHUB_NAME,transport_type = TransportType.AmqpOverWebsocket)
# producer = EventHubProducerClient.from_connection_string(conn_str = CONNECTION_STR, eventhub_name = EVENTHUB_NAME)
event_data_batch = producer.create_batch()
# msg_str = json.dumps(getClickStream())
msg_str = json.dumps(get_customer_transaction_Stream())
event_data_batch.add(EventData(msg_str)) # Add event data to the batch.
producer.send_batch(event_data_batch) # Send the batch of events to the event hub.

# Close the producer.
producer.close()

print("msg sent",msg_str)


msg sent {"transaction_id": "06e79ca0-0c1d-4712-91ae-5e243864083d", "store_number": 101, "customer_number": 28144, "product": "IceCream", "sold_quantity": 4, "customer_total_spent": 52, "customer_state": "WA", "transaction_time": "2022-09-04T04:33:32.347584"}


In [0]:
lv_event_hub_name = 'sales_topic'

In [0]:

event_hub_connection_string = "Endpoint=sb://zehubname1.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=xxxxxxxxxxxxxx=" # Paste your Event Hubs connection string in the quotes to the left
event_hub_name = "sales_topic"
connection_string = event_hub_connection_string + ";EntityPath=" + event_hub_name


import json

# Create the starting position Dictionary
startingEventPosition = {
  "offset": "-1",
  "seqNo": -1,            # not in use
  "enqueuedTime": None,   # not in use
  "isInclusive": True
}

eventHubsConf = {
  "eventhubs.connectionString" : sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_string),
  "eventhubs.startingPosition" : json.dumps(startingEventPosition),
  "setMaxEventsPerTrigger": 100
}

In [0]:
from pyspark.sql.functions import col

spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

eventStreamDF = (spark.readStream
  .format("eventhubs")
  .options(**eventHubsConf)
  .load()
)

eventStreamDF.printSchema()

root
 |-- body: binary (nullable = true)
 |-- partition: string (nullable = true)
 |-- offset: string (nullable = true)
 |-- sequenceNumber: long (nullable = true)
 |-- enqueuedTime: timestamp (nullable = true)
 |-- publisher: string (nullable = true)
 |-- partitionKey: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- systemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [0]:
# display(eventStreamDF)

In [0]:
staging_path = f"dbfs:/temp/{lv_event_hub_name}/staging"
dbutils.fs.mkdirs(staging_path)
staging_sink_location = f"{staging_path}/delta"
staging_checkpointPath = f"{staging_path}/write-checkpoint"
print(staging_sink_location)
# dbutils.fs.rm(checkpointPath,True)

dbfs:/temp/sales_topic/staging/delta


In [0]:
query_staging = (eventStreamDF.select(col("body").cast("STRING").alias("message"),col("properties"),col("enqueuedTime"))
  .writeStream
  .format("delta")
  .outputMode("append")
  .option("checkpointLocation", staging_checkpointPath)
  .trigger(once=True)
  .start(staging_sink_location))

query_staging.awaitTermination()

In [0]:
events_staging = spark.read.format("delta").load(f"{staging_sink_location}")
display(events_staging)

message,properties,enqueuedTime
"{""transaction_id"": ""5df81af5-78fe-4b11-b96d-6ca89352ea3c"", ""store_number"": 184, ""customer_number"": 26858, ""product"": ""Chips"", ""sold_quantity"": 1, ""customer_total_spent"": 81, ""customer_state"": ""NSW"", ""transaction_time"": ""2022-09-04T04:32:49.352448""}",Map(),2022-09-04T04:32:49.436+0000
"{""transaction_id"": ""95de9256-98c8-4575-aa29-b788d9077728"", ""store_number"": 159, ""customer_number"": 25988, ""product"": ""Chips"", ""sold_quantity"": 5, ""customer_total_spent"": 100, ""customer_state"": ""NT"", ""transaction_time"": ""2022-09-04T04:32:53.530292""}",Map(),2022-09-04T04:32:53.608+0000
"{""transaction_id"": ""120f0d69-31e3-4ee9-9c67-febbc66e0abf"", ""store_number"": 132, ""customer_number"": 28828, ""product"": ""IceCream"", ""sold_quantity"": 5, ""customer_total_spent"": 69, ""customer_state"": ""VIC"", ""transaction_time"": ""2022-09-04T04:32:57.887482""}",Map(),2022-09-04T04:32:57.967+0000
"{""transaction_id"": ""eba6ebe4-0251-4f22-88a4-8c5086fe82b5"", ""store_number"": 136, ""customer_number"": 21008, ""product"": ""Pizza"", ""sold_quantity"": 4, ""customer_total_spent"": 48, ""customer_state"": ""VIC"", ""transaction_time"": ""2022-09-04T04:33:02.211607""}",Map(),2022-09-04T04:33:02.296+0000
"{""transaction_id"": ""f1360f74-34c8-4ec9-a0f9-6a52408e4f96"", ""store_number"": 155, ""customer_number"": 28124, ""product"": ""Juice"", ""sold_quantity"": 2, ""customer_total_spent"": 22, ""customer_state"": ""QLD"", ""transaction_time"": ""2022-09-04T04:33:06.499927""}",Map(),2022-09-04T04:33:06.578+0000
"{""transaction_id"": ""0e0129e5-b01d-4862-ad5b-3729a5c93d03"", ""store_number"": 116, ""customer_number"": 24662, ""product"": ""Burger"", ""sold_quantity"": 5, ""customer_total_spent"": 71, ""customer_state"": ""NT"", ""transaction_time"": ""2022-09-04T04:33:10.834075""}",Map(),2022-09-04T04:33:10.907+0000
"{""transaction_id"": ""06e79ca0-0c1d-4712-91ae-5e243864083d"", ""store_number"": 101, ""customer_number"": 28144, ""product"": ""IceCream"", ""sold_quantity"": 4, ""customer_total_spent"": 52, ""customer_state"": ""WA"", ""transaction_time"": ""2022-09-04T04:33:32.347584""}",Map(),2022-09-04T04:33:32.438+0000
"{""transaction_id"": ""cc51d3df-8e13-467e-b8f1-55b92a167dc5"", ""store_number"": 161, ""customer_number"": 24335, ""product"": ""Chips"", ""sold_quantity"": 5, ""customer_total_spent"": 83, ""customer_state"": ""NSW"", ""transaction_time"": ""2022-09-04T02:33:35.877801""}",Map(),2022-09-04T02:33:36.106+0000
"{""transaction_id"": ""b13dac6e-5d68-4867-8730-0e5f3fe4f315"", ""store_number"": 169, ""customer_number"": 21032, ""product"": ""IceCream"", ""sold_quantity"": 3, ""customer_total_spent"": 81, ""customer_state"": ""WA"", ""transaction_time"": ""2022-09-04T02:33:43.801069""}",Map(),2022-09-04T02:33:43.888+0000
"{""transaction_id"": ""37956832-2850-4bab-aac0-2b222282b127"", ""store_number"": 176, ""customer_number"": 26802, ""product"": ""Juice"", ""sold_quantity"": 1, ""customer_total_spent"": 81, ""customer_state"": ""SA"", ""transaction_time"": ""2022-09-04T02:33:48.829170""}",Map(),2022-09-04T02:33:48.919+0000


In [0]:
events_staging = spark.read.format("delta").load(f"{staging_sink_location}").filter(col("message").contains("NSW"))
display(events_staging)

message,properties,enqueuedTime
"{""transaction_id"": ""5df81af5-78fe-4b11-b96d-6ca89352ea3c"", ""store_number"": 184, ""customer_number"": 26858, ""product"": ""Chips"", ""sold_quantity"": 1, ""customer_total_spent"": 81, ""customer_state"": ""NSW"", ""transaction_time"": ""2022-09-04T04:32:49.352448""}",Map(),2022-09-04T04:32:49.436+0000
"{""transaction_id"": ""cc51d3df-8e13-467e-b8f1-55b92a167dc5"", ""store_number"": 161, ""customer_number"": 24335, ""product"": ""Chips"", ""sold_quantity"": 5, ""customer_total_spent"": 83, ""customer_state"": ""NSW"", ""transaction_time"": ""2022-09-04T02:33:35.877801""}",Map(),2022-09-04T02:33:36.106+0000


##### delta lake staging layer to delta lake raw layer
##### incoming stream => split according to message type and flattned based json schema of message type and save as raw delta table for each message type

In [0]:
raw_path = f"dbfs:/temp/{lv_event_hub_name}/raw"
dbutils.fs.mkdirs(raw_path)
raw_sink_location = f"{raw_path}/delta"
raw_checkpointPath = f"{raw_path}/write-checkpoint"
print(raw_sink_location)
# dbutils.fs.rm(checkpointPath,True)

dbfs:/temp/sales_topic/raw/delta


In [0]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DoubleType
from pyspark.sql.functions import col, from_json

schema_raw1 = StructType([
  StructField("transaction_id", StringType(), True),
  StructField("store_number", StringType(), True),
  StructField("customer_number", StringType(), True),
  StructField("product", StringType(), True),
  StructField("sold_quantity", DoubleType(), True),
  StructField("customer_total_spent", DoubleType(), True),
  StructField("customer_state", StringType(), True),
  StructField("transaction_time", StringType(), True)
])

In [0]:
staging_df = spark.readStream.format("delta").load(staging_sink_location).select(col("message"))
flttend_df = staging_df.select(from_json(col("message"), schema_raw1).alias("message_data")).select("message_data.*")
query_raw = (flttend_df.writeStream
  .format("delta")
  .outputMode("append")
  .option("checkpointLocation", raw_checkpointPath)
  .trigger(once=True)
  .start(raw_sink_location))

query_raw.awaitTermination()

In [0]:
events_raw = spark.read.format("delta").load(f"{raw_sink_location}")
display(events_raw)

transaction_id,store_number,customer_number,product,sold_quantity,customer_total_spent,customer_state,transaction_time
cc51d3df-8e13-467e-b8f1-55b92a167dc5,161,24335,Chips,5.0,83.0,NSW,2022-09-04T02:33:35.877801
b13dac6e-5d68-4867-8730-0e5f3fe4f315,169,21032,IceCream,3.0,81.0,WA,2022-09-04T02:33:43.801069
37956832-2850-4bab-aac0-2b222282b127,176,26802,Juice,1.0,81.0,SA,2022-09-04T02:33:48.829170
2514212e-481f-4192-9622-b11da46c9f36,198,23019,IceCream,5.0,98.0,WA,2022-09-04T02:33:53.504886
94345856-3261-45dd-90b2-e2c859da59c6,142,29310,Chips,1.0,52.0,NT,2022-09-04T02:33:58.333144
c6856abc-02e8-4ec3-8817-ec20a7ecf8ec,104,26498,IceCream,3.0,92.0,VIC,2022-09-04T02:34:02.957051
7370c32e-6245-440f-8d84-d5af20c50096,112,26319,Pizza,1.0,63.0,VIC,2022-09-04T02:34:08.529944
5df81af5-78fe-4b11-b96d-6ca89352ea3c,184,26858,Chips,1.0,81.0,NSW,2022-09-04T04:32:49.352448
95de9256-98c8-4575-aa29-b788d9077728,159,25988,Chips,5.0,100.0,NT,2022-09-04T04:32:53.530292
120f0d69-31e3-4ee9-9c67-febbc66e0abf,132,28828,IceCream,5.0,69.0,VIC,2022-09-04T04:32:57.887482


##### delta lake raw layer to delta lake curated layer

In [0]:
curated_path = f"dbfs:/temp/{lv_event_hub_name}/curated"
dbutils.fs.mkdirs(curated_path)
curated_sink_location = f"{curated_path}/delta"
curated_checkpointPath = f"{curated_path}/write-checkpoint"
print(curated_sink_location)
# dbutils.fs.rm(checkpointPath,True)

dbfs:/temp/sales_topic/curated/delta


In [0]:
from pyspark.sql.functions import lit, current_timestamp, to_timestamp, current_date
column_list = ['transaction_id', 'store_number', 'customer_number', 'product', 'sold_quantity', 'customer_total_spent', 'customer_state', 'transaction_time']
raw_df = spark.readStream.format("delta").load(raw_sink_location)
transformed_df = raw_df.select(*column_list).withColumn("load_date", current_date()).withColumn("load_time", current_timestamp()).withColumn("data_source", lit(lv_event_hub_name)) 
query_curated= (transformed_df.writeStream
  .format("delta")
  .outputMode("append")
  .option("checkpointLocation", curated_checkpointPath)
  .trigger(once=True)
  .start(curated_sink_location))

query_curated.awaitTermination()

In [0]:
events_curated = spark.read.format("delta").load(f"{curated_sink_location}")
display(events_curated)

transaction_id,store_number,customer_number,product,sold_quantity,customer_total_spent,customer_state,transaction_time,load_date,load_time,data_source
cc51d3df-8e13-467e-b8f1-55b92a167dc5,161,24335,Chips,5.0,83.0,NSW,2022-09-04T02:33:35.877801,2022-09-04,2022-09-04T04:29:13.497+0000,sales_topic
b13dac6e-5d68-4867-8730-0e5f3fe4f315,169,21032,IceCream,3.0,81.0,WA,2022-09-04T02:33:43.801069,2022-09-04,2022-09-04T04:29:13.497+0000,sales_topic
37956832-2850-4bab-aac0-2b222282b127,176,26802,Juice,1.0,81.0,SA,2022-09-04T02:33:48.829170,2022-09-04,2022-09-04T04:29:13.497+0000,sales_topic
2514212e-481f-4192-9622-b11da46c9f36,198,23019,IceCream,5.0,98.0,WA,2022-09-04T02:33:53.504886,2022-09-04,2022-09-04T04:29:13.497+0000,sales_topic
94345856-3261-45dd-90b2-e2c859da59c6,142,29310,Chips,1.0,52.0,NT,2022-09-04T02:33:58.333144,2022-09-04,2022-09-04T04:29:13.497+0000,sales_topic
c6856abc-02e8-4ec3-8817-ec20a7ecf8ec,104,26498,IceCream,3.0,92.0,VIC,2022-09-04T02:34:02.957051,2022-09-04,2022-09-04T04:29:13.497+0000,sales_topic
7370c32e-6245-440f-8d84-d5af20c50096,112,26319,Pizza,1.0,63.0,VIC,2022-09-04T02:34:08.529944,2022-09-04,2022-09-04T04:29:13.497+0000,sales_topic
5df81af5-78fe-4b11-b96d-6ca89352ea3c,184,26858,Chips,1.0,81.0,NSW,2022-09-04T04:32:49.352448,2022-09-04,2022-09-04T04:33:52.401+0000,sales_topic
95de9256-98c8-4575-aa29-b788d9077728,159,25988,Chips,5.0,100.0,NT,2022-09-04T04:32:53.530292,2022-09-04,2022-09-04T04:33:52.401+0000,sales_topic
120f0d69-31e3-4ee9-9c67-febbc66e0abf,132,28828,IceCream,5.0,69.0,VIC,2022-09-04T04:32:57.887482,2022-09-04,2022-09-04T04:33:52.401+0000,sales_topic


In [0]:
dbutils.notebook.exit("finished")

finished

## Writing to Multiple Tables

Those familiar with Structured Streaming may be aware that the **`foreachBatch`** method provides the option to execute custom data writing logic on each microbatch of streaming data.

The Databricks Runtime provides guarantees that these <a href="https://docs.databricks.com/delta/delta-streaming.html#idempot-write" target="_blank">streaming Delta Lake writes will be idempotent</a>, even when writing to multiple tables, IF you set the "txnVersion" and "txnAppId" options. This is especially useful when data for multiple tables might be contained within a single record.  This was added in <a href="https://docs.databricks.com/release-notes/runtime/8.4.html" target="_blank">Databricks Runtime 8.4</a>.

The code below first defines the custom writer logic to append records to two new tables, and then demonstrates using this function within **`foreachBatch`**.

There is some debate as to whether you should use foreachBatch to write to multiple tables or to simply use multiple streams.  Generally multiple streams is the simpler and more efficient design because it allows streaming jobs writing to each table to run independently of each other.  Whereas using foreachBatch to write to multiple tables has the advantage of keeping writes to the two tables in sync.

In [0]:
#   // Appending to Path
#   Sampledata.write.format("delta").mode("append").save("/delta/events")
# raw_path = 'dbfs:/temp/sales_topic/raw'

raw_sink_message_type1 = f"{raw_path}/NSW/delta"
raw_sink_message_type2 = f"{raw_path}/VIC/delta"
raw_sink_message_type3 = f"{raw_path}/EXTRA/delta"
raw_multi_checkpoint = "dbfs:/temp/sales_topic/raw/split_stream/checkpoint"

from pyspark.sql.types import StructField, StructType, StringType, LongType, DoubleType
from pyspark.sql.functions import col, from_json

message1_schema = StructType([
  StructField("transaction_id", StringType(), True),
  StructField("store_number", StringType(), True),
  StructField("customer_number", StringType(), True),
  StructField("product", StringType(), True),
  StructField("sold_quantity", DoubleType(), True),
  StructField("customer_total_spent", DoubleType(), True),
  StructField("customer_state", StringType(), True),
  StructField("transaction_time", StringType(), True)
])

def write_multiple_target(microBatchDF, batchId):
    appId = "write_twice"
    
    message_type_df1 = microBatchDF.filter(col("message").contains("NSW")).select(col("message"))
    flttend_msg_df1 = message_type_df1.select(from_json(col("message"), message1_schema).alias("message_data")).select("message_data.*")
    flttend_msg_df1.write.format('delta').mode('append').save(raw_sink_message_type1)
    
    message_type_df2 = microBatchDF.filter(col("message").contains("VIC")).select(col("message"))
    flttend_msg_df2 = message_type_df2.select(from_json(col("message"), message1_schema).alias("message_data")).select("message_data.*")
    flttend_msg_df2.write.format('delta').mode('append').save(raw_sink_message_type2)

    message_type_df3 = microBatchDF.filter(~col("message").contains("VIC") & ~col("message").contains("NSW")).select(col("message"))
    flttend_msg_df3 = message_type_df3.select(from_json(col("message"), message1_schema).alias("message_data")).select("message_data.*")
    flttend_msg_df3.write.format('delta').mode('append').save(raw_sink_message_type3)


def split_stream():
    query = (spark.readStream
                   .format("delta")
                   .load(staging_sink_location)
                   .writeStream
                   .foreachBatch(write_multiple_target)
                   .option("checkpointLocation", raw_multi_checkpoint)
                   .trigger(once=True)
                   .start())

    query.awaitTermination()


In [0]:
%fs ls dbfs:/temp/sales_topic/raw/

path,name,size,modificationTime
dbfs:/temp/sales_topic/raw/EXTRA/,EXTRA/,0,0
dbfs:/temp/sales_topic/raw/NSW/,NSW/,0,0
dbfs:/temp/sales_topic/raw/VIC/,VIC/,0,0
dbfs:/temp/sales_topic/raw/delta/,delta/,0,0
dbfs:/temp/sales_topic/raw/split_stream/,split_stream/,0,0
dbfs:/temp/sales_topic/raw/write-checkpoint/,write-checkpoint/,0,0


Note that while a stream will again be triggered, the multiple writes contained within the **`write_multiple_target`** function are using Spark batch syntax. This will always be the case for writers called by **`foreachBatch`**.

In [0]:
split_stream()

In [0]:
events_raw_df = spark.read.format("delta").load(f"{raw_sink_message_type3}")
display(events_raw_df)


transaction_id,store_number,customer_number,product,sold_quantity,customer_total_spent,customer_state,transaction_time
b13dac6e-5d68-4867-8730-0e5f3fe4f315,169,21032,IceCream,3.0,81.0,WA,2022-09-04T02:33:43.801069
37956832-2850-4bab-aac0-2b222282b127,176,26802,Juice,1.0,81.0,SA,2022-09-04T02:33:48.829170
2514212e-481f-4192-9622-b11da46c9f36,198,23019,IceCream,5.0,98.0,WA,2022-09-04T02:33:53.504886
94345856-3261-45dd-90b2-e2c859da59c6,142,29310,Chips,1.0,52.0,NT,2022-09-04T02:33:58.333144
95de9256-98c8-4575-aa29-b788d9077728,159,25988,Chips,5.0,100.0,NT,2022-09-04T04:32:53.530292
f1360f74-34c8-4ec9-a0f9-6a52408e4f96,155,28124,Juice,2.0,22.0,QLD,2022-09-04T04:33:06.499927
0e0129e5-b01d-4862-ad5b-3729a5c93d03,116,24662,Burger,5.0,71.0,NT,2022-09-04T04:33:10.834075
06e79ca0-0c1d-4712-91ae-5e243864083d,101,28144,IceCream,4.0,52.0,WA,2022-09-04T04:33:32.347584
f8e6a6c1-0784-482c-9f20-545fd9074ad8,148,22337,Pizza,5.0,52.0,QLD,2022-09-04T04:28:50.130574


### write multiple curated stream target

In [0]:
from pyspark.sql import functions as F

def stream_from_to_delta_lake(source_delta,sink_delta):
  sink_delta_data = f"{sink_delta}/data"
  sink_delta_checkpoint = f"{sink_delta}/checkpoint"
  source_stream_df = spark.readStream.format("delta").load(source_delta)
  tranform_stream_df = source_stream_df.withColumn("load_date", current_date()).withColumn("load_time", current_timestamp()).withColumn("data_source", lit(lv_event_hub_name)) 
  stream_query = (tranform_stream_df.writeStream
                        .format("delta")
                        .outputMode("append")
                        .option("checkpointLocation", sink_delta_checkpoint)
                        .trigger(once=True)
                        .start(sink_delta_data)   )
  
  stream_query.awaitTermination()
 

In [0]:
source_raw_delta = "dbfs:/temp/sales_topic/raw/VIC/delta"
sink_curated_delta = "dbfs:/temp/sales_topic/curated/VIC"
stream_from_to_delta_lake(source_raw_delta,sink_curated_delta)

source_raw_delta = "dbfs:/temp/sales_topic/raw/NSW/delta"
sink_curated_delta = "dbfs:/temp/sales_topic/curated/NSW"
stream_from_to_delta_lake(source_raw_delta,sink_curated_delta)

source_raw_delta = "dbfs:/temp/sales_topic/raw/EXTRA/delta"
sink_curated_delta = "dbfs:/temp/sales_topic/curated/EXTRA"
stream_from_to_delta_lake(source_raw_delta,sink_curated_delta)


In [0]:
%fs ls dbfs:/temp/sales_topic/curated/

path,name,size,modificationTime
dbfs:/temp/sales_topic/curated/EXTRA/,EXTRA/,0,0
dbfs:/temp/sales_topic/curated/NSW/,NSW/,0,0
dbfs:/temp/sales_topic/curated/VIC/,VIC/,0,0


In [0]:
events_curated_df = spark.read.format("delta").load(f"dbfs:/temp/sales_topic/curated/EXTRA/data")
display(events_curated_df)


transaction_id,store_number,customer_number,product,sold_quantity,customer_total_spent,customer_state,transaction_time,load_date,load_time,data_source
b13dac6e-5d68-4867-8730-0e5f3fe4f315,169,21032,IceCream,3.0,81.0,WA,2022-09-04T02:33:43.801069,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic
37956832-2850-4bab-aac0-2b222282b127,176,26802,Juice,1.0,81.0,SA,2022-09-04T02:33:48.829170,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic
2514212e-481f-4192-9622-b11da46c9f36,198,23019,IceCream,5.0,98.0,WA,2022-09-04T02:33:53.504886,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic
94345856-3261-45dd-90b2-e2c859da59c6,142,29310,Chips,1.0,52.0,NT,2022-09-04T02:33:58.333144,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic
95de9256-98c8-4575-aa29-b788d9077728,159,25988,Chips,5.0,100.0,NT,2022-09-04T04:32:53.530292,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic
f1360f74-34c8-4ec9-a0f9-6a52408e4f96,155,28124,Juice,2.0,22.0,QLD,2022-09-04T04:33:06.499927,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic
0e0129e5-b01d-4862-ad5b-3729a5c93d03,116,24662,Burger,5.0,71.0,NT,2022-09-04T04:33:10.834075,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic
06e79ca0-0c1d-4712-91ae-5e243864083d,101,28144,IceCream,4.0,52.0,WA,2022-09-04T04:33:32.347584,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic
f8e6a6c1-0784-482c-9f20-545fd9074ad8,148,22337,Pizza,5.0,52.0,QLD,2022-09-04T04:28:50.130574,2022-09-04,2022-09-04T04:39:27.049+0000,sales_topic


The **`MERGE`** statement can easily be written with SQL to apply CDC changes appropriately, given the type of update received.

The rest of the **`upsert_cdc`** method contains the logic necessary to run SQL code against a micro-batch in a PySpark DataStreamWriter.

In [0]:
def upsert_cdc(microBatchDF, batchID):
    microBatchDF.createTempView("bronze_batch")
    
    query = """
        MERGE INTO silver_status s
        USING bronze_batch b
        ON b.user_id = s.user_id
        WHEN MATCHED AND b.update_type = "update"
          THEN UPDATE SET user_id=b.user_id, status=b.status, updated_timestamp=b.processed_timestamp
        WHEN MATCHED AND b.update_type = "delete"
          THEN DELETE
        WHEN NOT MATCHED AND b.update_type = "update" OR b.update_type = "insert"
          THEN INSERT (user_id, status, updated_timestamp)
          VALUES (b.user_id, b.status, b.processed_timestamp)
    """
    
    microBatchDF._jdf.sparkSession().sql(query)
    
def streaming_merge():
    query = (spark.readStream
                  .table("bronze_status")
                  .writeStream
                  .foreachBatch(upsert_cdc)
                  .option("checkpointLocation", f"{DA.paths.checkpoints}/silver_status")
                  .trigger(availableNow=True)
                  .start())
    
    query.awaitTermination()
    

As always, we incrementally process newly arriving records.

In [0]:
streaming_merge()